In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/restaurants-dataset/foody_final_data/foody_final_data/foody_reviews.csv
/kaggle/input/restaurants-dataset/foody_final_data/foody_final_data/foody_restaurants.csv
/kaggle/input/restaurants-dataset/befood_data/restaurants.csv
/kaggle/input/restaurants-dataset/befood_data/dishes.csv
/kaggle/input/restaurants-dataset/befood_data/reviews.csv
/kaggle/input/restaurants-dataset/foody_data/restaurants.csv
/kaggle/input/restaurants-dataset/foody_data/dishes.csv
/kaggle/input/restaurants-dataset/foody_data/reviews.csv
/kaggle/input/restaurants-dataset/befood_final_data/befood_final_data/befood_reviews.csv
/kaggle/input/restaurants-dataset/befood_final_data/befood_final_data/befood_dishes.csv
/kaggle/input/restaurants-dataset/befood_final_data/befood_final_data/befood_restaurants.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from datetime import datetime, timedelta

In [3]:
output_dir = '/kaggle/working/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Be food

In [4]:
befood_restaurants = pd.read_csv('/kaggle/input/restaurants-dataset/befood_final_data/befood_final_data/befood_restaurants.csv')
befood_reviews = pd.read_csv('/kaggle/input/restaurants-dataset/befood_final_data/befood_final_data/befood_reviews.csv')
befood_dishes = pd.read_csv('/kaggle/input/restaurants-dataset/befood_final_data/befood_final_data/befood_dishes.csv')

/tmp/ipykernel_19/3052550450.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  befood_dishes = pd.read_csv('/kaggle/input/restaurants-dataset/befood_final_data/befood_final_data/befood_dishes.csv')


In [5]:
befood_restaurants

,restaurant_id,restaurant_name,latitude,longitude,address,restaurant_rating,review_count,city_id,platform_id,district_mapped,district_id
0,1.0,Cơm Cháy Tròn,10.79690,106.694641,"87/30 Lê Văn Duyệt, Phường 3, Quận Bình Thạnh",5.0,1.0,189.0,0,quận bình thạnh,16
1,2.0,bánh tráng trộn,10.77513,106.725700,"10 Mai Chí Thọ, An Lợi Đông, Quận 2, TP Hồ Chí...",0.0,0.0,189.0,0,quận 2,6
2,3.0,Tiệm Tạp hoá Nhà Trang,10.79196,106.712930,"76 Ngô Tất Tố, Phường 19, Quận Bình Thạnh",0.0,0.0,189.0,0,quận bình thạnh,16
3,4.0,Runi,10.78554,106.698530,154/3 LCT,5.0,4.0,189.0,0,quận 1,2
4,5.0,Mẹ Cherry - Đồ Ăn Cho Bé,10.80427,106.696290,"104 Trần Văn Kỷ, Phường 14, Q.Bình Thạnh",0.0,0.0,189.0,0,quận bình thạnh,16
...,...,...,...,...,...,...,...,...,...,...,...
102163,109959.0,Trộn Nó Lên - Mì Trộn & Sandwich - Trương Định,20.98988,105.848170,"20 Ngõ 207 Đường Trương Định, Phường Tương Mai...",0.0,0.0,189.0,0,quận hoàng mai,26
102164,109963.0,Bánh Cuốn Tuấn Hằng - Hưng Phú,10.74655,106.670500,"621 Hưng Phú, Phường 9, Quận 8, Thành Phố Hồ C...",0.0,0.0,189.0,0,quận 8,12
102165,109966.0,Mít Food - Bún Đậu Mắm Tôm - Ngõ 374 Trần Khát...,21.00957,105.857400,"Số 2A Ngõ 374 Trần Khát Chân, Phường Bạch Đằng...",0.0,0.0,189.0,0,quận hai bà trưng,23
102166,109971.0,Meo Food - Mỳ Trộn Indomie - Láng Trung,21.01386,105.808420,"79 Ngõ Láng Trung, Phường Láng Hạ, Quận Đống Đ...",0.0,0.0,189.0,0,quận đống đa,40


In [6]:
def clean_befood_restaurants(df):
    import pandas as pd

    df_cleaned = df.copy()
    # drop null
    df_cleaned = df_cleaned.dropna()
    # Đổi tên cột
    df_cleaned = df_cleaned.rename(columns={'district_mapped': 'district_name'})

    # Chuyển kiểu dữ liệu
    df_cleaned['restaurant_id'] = df_cleaned['restaurant_id'].astype(str)
    df_cleaned['restaurant_rating'] = pd.to_numeric(df_cleaned['restaurant_rating'], errors='coerce')
    df_cleaned['review_count'] = pd.to_numeric(df_cleaned['review_count'], errors='coerce').fillna(0).astype(int)

    # Xoá city_id nếu có
    if 'city_id' in df_cleaned.columns:
        df_cleaned = df_cleaned.drop(columns=['city_id'])

    # Chuẩn hoá district_name
    district_name_corrections = {
        'quận bình chánh': 'huyện bình chánh',
        'quận hóc môn': 'huyện hóc môn',
        'quận nhà bè': 'huyện nhà bè',
        'quận hoài đức': 'huyện hoài đức',
        'quận gia lâm': 'huyện gia lâm',
        'quận thanh trì': 'huyện thanh trì',
        'quận từ liêm': 'quận nam từ liêm',  # Gộp nếu cần
        'quận tân quý': None,
        'quận dương nội': None,
        'Quận Hà Nội': None
    }

    df_cleaned['district_name'] = df_cleaned['district_name'].replace(district_name_corrections)
    df_cleaned = df_cleaned[df_cleaned['district_name'].notna()].copy()

    # Mapping district → id
    district_id_mapping = {
        'quận 1': 2, 'quận 2': 6, 'quận 3': 7, 'quận 4': 8, 'quận 5': 9, 'quận 6': 10,
        'quận 7': 11, 'quận 8': 12, 'quận 9': 13, 'quận 10': 3, 'quận 11': 4, 'quận 12': 5,
        'quận bình thạnh': 16, 'quận gò vấp': 22, 'quận phú nhuận': 32, 'quận tân bình': 35,
        'quận tân phú': 36, 'quận bình tân': 17, 'thành phố thủ đức': 41, 'huyện bình chánh': 15,
        'huyện hóc môn': 28, 'huyện nhà bè': 31,
        'quận ba đình': 14, 'quận hoàng mai': 26, 'quận hai bà trưng': 23, 'quận đống đa': 40,
        'quận hoàn kiếm': 25, 'quận cầu giấy': 19, 'quận tây hồ': 38, 'quận long biên': 29,
        'quận thanh xuân': 34, 'quận hà đông': 27, 'quận bắc từ liêm': 18, 'quận nam từ liêm': 30,
        'huyện gia lâm': 21, 'huyện hoài đức': 24, 'huyện thanh trì': 33, 'huyện thạch thất': 1
    }

    df_cleaned['district_id'] = df_cleaned['district_name'].map(district_id_mapping)

    # Danh sách quận TP. HCM và Hà Nội
    hcm_districts = [
        'quận 1', 'quận 2', 'quận 3', 'quận 4', 'quận 5', 'quận 6', 'quận 7', 'quận 8', 'quận 9',
        'quận 10', 'quận 11', 'quận 12', 'quận bình thạnh', 'quận gò vấp', 'quận phú nhuận',
        'quận tân bình', 'quận tân phú', 'quận bình tân', 'thành phố thủ đức',
        'huyện bình chánh', 'huyện hóc môn', 'huyện nhà bè'
    ]

    hanoi_districts = [
        'quận ba đình', 'quận hoàng mai', 'quận hai bà trưng', 'quận đống đa', 'quận hoàn kiếm',
        'quận cầu giấy', 'quận tây hồ', 'quận long biên', 'quận thanh xuân', 'quận hà đông',
        'quận bắc từ liêm', 'quận nam từ liêm', 'huyện hoài đức', 'huyện thanh trì',
        'huyện gia lâm', 'huyện thạch thất'
    ]

    # Hàm gán city name
    def get_city_name(district):
        if district in hcm_districts:
            return 'TP. Hồ Chí Minh'
        elif district in hanoi_districts:
            return 'Hà Nội'
        else:
            return 'Khác'

    df_cleaned['city_name'] = df_cleaned['district_name'].apply(get_city_name)

    # Gán city_id
    city_id_map = {name: idx+1 for idx, name in enumerate(df_cleaned['city_name'].unique())}
    df_cleaned['city_id'] = df_cleaned['city_name'].map(city_id_map)

    return df_cleaned
befood_restaurants_cleaned = clean_befood_restaurants(befood_restaurants)

In [7]:
district_mapping = dict(befood_restaurants_cleaned[['district_name', 'district_id']].drop_duplicates().values)
sorted_district_mapping = dict(sorted(district_mapping.items(), key=lambda x: x[1]))
sorted_district_mapping

{'huyện thạch thất': 1,
 'quận 1': 2,
 'quận 10': 3,
 'quận 11': 4,
 'quận 12': 5,
 'quận 2': 6,
 'quận 3': 7,
 'quận 4': 8,
 'quận 5': 9,
 'quận 6': 10,
 'quận 7': 11,
 'quận 8': 12,
 'quận 9': 13,
 'quận ba đình': 14,
 'huyện bình chánh': 15,
 'quận bình thạnh': 16,
 'quận bình tân': 17,
 'quận bắc từ liêm': 18,
 'quận cầu giấy': 19,
 'huyện gia lâm': 21,
 'quận gò vấp': 22,
 'quận hai bà trưng': 23,
 'huyện hoài đức': 24,
 'quận hoàn kiếm': 25,
 'quận hoàng mai': 26,
 'quận hà đông': 27,
 'huyện hóc môn': 28,
 'quận long biên': 29,
 'quận nam từ liêm': 30,
 'huyện nhà bè': 31,
 'quận phú nhuận': 32,
 'huyện thanh trì': 33,
 'quận thanh xuân': 34,
 'quận tân bình': 35,
 'quận tân phú': 36,
 'quận tây hồ': 38,
 'quận đống đa': 40,
 'thành phố thủ đức': 41}

In [8]:
befood_reviews

,rating_id,restaurant_id,user_name,rating,review_text,review_time,user_id,platform_id
0,1,1,Minh Vũ,5,"Ngon xỉu,Đáng đồng tiền,Sạch sẽ,Đóng gói tốt,Khác",17:03 03/11/2021,1,0
1,1315266,4,Tina,5,"cũng được nha, Ngon xỉu, Đáng đồng tiền, Sạch ...",10:56 28/04/2023,2,0
2,1314314,4,Tina,5,"Đáng đồng tiền, Làm món nhanh",01:04 28/04/2023,2,0
3,1315588,4,Nhi,5,NaN,11:37 28/04/2023,3,0
4,573399,4,Kim Nguyen,5,NaN,23:24 12/10/2022,4,0
...,...,...,...,...,...,...,...,...
7464862,7575828,109664,Hằng Hà,5,NaN,14 phút trước,83983,0
7464863,7575295,109711,Diệu Lê,5,"vị chua ngọt vừa,, Ngon xỉu, Đóng gói tốt, Sạc...",1 giờ trước,34687,0
7464864,7570679,109712,Nguyenminh Khoa,5,"Ngon xỉu, Đáng đồng tiền, Sạch sẽ, Đóng gói tốt",6 giờ trước,157637,0
7464865,7573475,109730,Thiên Ân,5,"Ngon xỉu, Đáng đồng tiền, Sạch sẽ, Đóng gói tốt",3 giờ trước,1606,0


In [9]:
import pandas as pd
import re
from datetime import datetime, timedelta

def clean_befood_reviews(df):
    # Tạo bản sao để không ảnh hưởng đến dữ liệu gốc
    df_cleaned = df.copy()
    
    # Tạo user_id 
    df_cleaned['user_id'] = 'BU_' + df_cleaned['user_id'].astype(str)
    
    # Đảm bảo dữ liệu có kiểu dữ liệu phù hợp
    df_cleaned['restaurant_id'] = df_cleaned['restaurant_id'].astype(str)
    df_cleaned['rating_id'] = df_cleaned['rating_id'].astype(str)
    df_cleaned['rating'] = pd.to_numeric(df_cleaned['rating'], errors='coerce')
    
    # Định nghĩa thời điểm thu thập dữ liệu - điều chỉnh nếu cần
    collection_time = datetime.now()
    
    # Hàm xử lý review_time để chuyển về định dạng dd/mm/yyyy
    def convert_befood_time(time_str):
        if pd.isna(time_str):
            return None
        
        time_str = str(time_str)
        
        # Định dạng "HH:MM DD/MM/YYYY" -> "DD/MM/YYYY"
        if ':' in time_str and '/' in time_str:
            try:
                # Trích xuất chỉ phần ngày tháng năm
                date_part = time_str.split(' ')[1]
                return date_part  # Đã ở định dạng DD/MM/YYYY
            except:
                pass
        
        # Xử lý thời gian tương đối
        try:
            result_date = None
            if 'phút trước' in time_str:
                minutes = int(re.search(r'(\d+)', time_str).group(1))
                result_date = collection_time - timedelta(minutes=minutes)
            elif 'giờ trước' in time_str:
                hours = int(re.search(r'(\d+)', time_str).group(1))
                result_date = collection_time - timedelta(hours=hours)
            elif 'ngày trước' in time_str:
                days = int(re.search(r'(\d+)', time_str).group(1))
                result_date = collection_time - timedelta(days=days)
            elif 'tuần trước' in time_str:
                weeks = int(re.search(r'(\d+)', time_str).group(1))
                result_date = collection_time - timedelta(weeks=weeks)
            elif 'tháng trước' in time_str:
                months = int(re.search(r'(\d+)', time_str).group(1))
                result_date = collection_time - timedelta(days=30*months)
                
            if result_date:
                return result_date.strftime('%d/%m/%Y')
        except:
            pass
            
        # Nếu là các định dạng ngày tháng khác, cố gắng chuyển đổi
        try:
            dt = pd.to_datetime(time_str)
            return dt.strftime('%d/%m/%Y')
        except:
            return None
    
    # Áp dụng hàm xử lý thời gian
    df_cleaned['review_time'] = df_cleaned['review_time'].apply(convert_befood_time)
    
    return df_cleaned 

In [10]:
befood_reviews_cleaned = clean_befood_reviews(befood_reviews)

In [11]:
befood_reviews_cleaned

,rating_id,restaurant_id,user_name,rating,review_text,review_time,user_id,platform_id
0,1,1,Minh Vũ,5,"Ngon xỉu,Đáng đồng tiền,Sạch sẽ,Đóng gói tốt,Khác",03/11/2021,BU_1,0
1,1315266,4,Tina,5,"cũng được nha, Ngon xỉu, Đáng đồng tiền, Sạch ...",28/04/2023,BU_2,0
2,1314314,4,Tina,5,"Đáng đồng tiền, Làm món nhanh",28/04/2023,BU_2,0
3,1315588,4,Nhi,5,NaN,28/04/2023,BU_3,0
4,573399,4,Kim Nguyen,5,NaN,12/10/2022,BU_4,0
...,...,...,...,...,...,...,...,...
7464862,7575828,109664,Hằng Hà,5,NaN,03/05/2025,BU_83983,0
7464863,7575295,109711,Diệu Lê,5,"vị chua ngọt vừa,, Ngon xỉu, Đóng gói tốt, Sạc...",03/05/2025,BU_34687,0
7464864,7570679,109712,Nguyenminh Khoa,5,"Ngon xỉu, Đáng đồng tiền, Sạch sẽ, Đóng gói tốt",03/05/2025,BU_157637,0
7464865,7573475,109730,Thiên Ân,5,"Ngon xỉu, Đáng đồng tiền, Sạch sẽ, Đóng gói tốt",03/05/2025,BU_1606,0


In [12]:
befood_dishes

,restaurant_id,category_id,category_name,dish_id,item_name,price,platform_id
0,1,2.0,MUA 1 TẶNG 1 CƠM 9K,15.0,Cơm cháy chà bông 500g,120000.0,0
1,1,2.0,MUA 1 TẶNG 1 CƠM 9K,16.0,Cơm cháy chà bông 250g,60000.0,0
2,1,2.0,MUA 1 TẶNG 1 CƠM 9K,17.0,Cơm cháy mắm hành 500g,105000.0,0
3,2,92430.0,Món Đang Giảm,708224.0,Trà Sữa Phô Mai Tươi,29000.0,0
4,2,92430.0,Món Đang Giảm,708225.0,Trà Sữa Bánh Flan,29000.0,0
...,...,...,...,...,...,...,...
3064619,109980,1340037.0,Mỳ Cay Đẫm Lệ,9175878.0,Mỳ Cay Xúc Xích—Cá Viên,55000.0,0
3064620,109980,1340037.0,Mỳ Cay Đẫm Lệ,9177485.0,Mỳ Cay Rau Nấm,40000.0,0
3064621,109980,1340037.0,Mỳ Cay Đẫm Lệ,9177588.0,Mỳ Cay Ba Chỉ Bò,65000.0,0
3064622,109980,1340037.0,Mỳ Cay Đẫm Lệ,9177640.0,Mỳ Cay Hải Sản,70000.0,0


In [13]:
def clean_befood_dishes(df):
    df_cleaned = df.copy()
    # Đảm bảo dữ liệu có kiểu dữ liệu phù hợp
    df_cleaned['restaurant_id'] = df_cleaned['restaurant_id'].astype(str)
    df_cleaned['dish_id'] = df_cleaned['dish_id'].astype(str)
    df_cleaned['category_id'] = df_cleaned['category_id'].astype(str)
    df_cleaned['price'] = pd.to_numeric(df_cleaned['price'], errors='coerce')
    
    return df_cleaned

In [14]:
befood_dishes_cleaned = clean_befood_dishes(befood_dishes)
befood_dishes_cleaned

,restaurant_id,category_id,category_name,dish_id,item_name,price,platform_id
0,1,2.0,MUA 1 TẶNG 1 CƠM 9K,15.0,Cơm cháy chà bông 500g,120000.0,0
1,1,2.0,MUA 1 TẶNG 1 CƠM 9K,16.0,Cơm cháy chà bông 250g,60000.0,0
2,1,2.0,MUA 1 TẶNG 1 CƠM 9K,17.0,Cơm cháy mắm hành 500g,105000.0,0
3,2,92430.0,Món Đang Giảm,708224.0,Trà Sữa Phô Mai Tươi,29000.0,0
4,2,92430.0,Món Đang Giảm,708225.0,Trà Sữa Bánh Flan,29000.0,0
...,...,...,...,...,...,...,...
3064619,109980,1340037.0,Mỳ Cay Đẫm Lệ,9175878.0,Mỳ Cay Xúc Xích—Cá Viên,55000.0,0
3064620,109980,1340037.0,Mỳ Cay Đẫm Lệ,9177485.0,Mỳ Cay Rau Nấm,40000.0,0
3064621,109980,1340037.0,Mỳ Cay Đẫm Lệ,9177588.0,Mỳ Cay Ba Chỉ Bò,65000.0,0
3064622,109980,1340037.0,Mỳ Cay Đẫm Lệ,9177640.0,Mỳ Cay Hải Sản,70000.0,0


In [15]:
befood_dishes_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3064624 entries, 0 to 3064623
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   restaurant_id  object 
 1   category_id    object 
 2   category_name  object 
 3   dish_id        object 
 4   item_name      object 
 5   price          float64
 6   platform_id    int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 163.7+ MB


# Foody

In [16]:
foody_restaurants = pd.read_csv('/kaggle/input/restaurants-dataset/foody_final_data/foody_final_data/foody_restaurants.csv')
foody_reviews = pd.read_csv('/kaggle/input/restaurants-dataset/foody_final_data/foody_final_data/foody_reviews.csv')
foody_dishes = pd.read_csv('/kaggle/input/restaurants-dataset/foody_data/dishes.csv')

In [17]:
foody_restaurants = pd.read_csv('/kaggle/input/restaurants-dataset/foody_final_data/foody_final_data/foody_restaurants.csv')
foody_restaurants

,restaurant_id,restaurant_name,latitude,longitude,address,restaurant_rating,review_count,platform_id,city,district_raw
0,2,Lẩu Nấm Thiên Nhiên Ashima - Nguyễn Đình Chiểu...,10.786070,106.697114,"35A Nguyễn Đình Chiểu, P. Đa Kao, Quận 1, TP. HCM",4.0,176,1,TP. HCM,Quận 1
1,3,Nhung - Điện Biên - Test,21.383385,103.015743,"Số 851, Đường Võ Nguyên Giáp, Phường Mường Tha...",4.0,132,1,Điện Biên,Tp. Điện Biên Phủ
2,7,Nhung qa test - Test - 1,1.000000,2.000000,"12 Nguyễn Đình Chiểu, Phường Phạm Ngũ Lão, Quậ...",4.0,111,1,TP. HCM,Quận 1
3,8,Buffet Hải Sản - Café Central An Đông - Khách ...,10.757467,106.673387,"Windsor Plaza, 18 An Dương Vương, Quận 5, TP. HCM",4.0,189,1,TP. HCM,Quận 5
4,11,Honey Moon - Nhà Hàng Tiệc Cưới,10.760614,106.699829,"Tầng 2, Cao ốc H3, Hoàng Diệu, Phường 6, Quận ...",4.0,10,1,TP. HCM,Quận 4
...,...,...,...,...,...,...,...,...,...,...
31163,199985,Lò Bánh Mì Lương Oanh,16.479240,107.568374,"98 Thái Phiên, P. Tây Lộc, Tp. Huế, Huế",4.0,2,1,Huế,Tp. Huế
31164,199992,Swiss - Bel Resort Tuyen Lam Dalat,11.883076,108.451533,"Khu 7 - 8 Hồ Tuyền Lâm, P. 3, Thành Phố Đà Lạt...",4.0,44,1,Lâm Đồng,Thành Phố Đà Lạt
31165,199993,Siêu Thị Hàng Nhật,16.487564,107.587503,"121 Nguyễn Chí Thanh, Tp. Huế, Huế",4.0,1,1,Huế,Tp. Huế
31166,199995,Bún Mắm Nêm Khuya,10.940482,106.863754,"Trần Quốc Toản, P. Bình Đa, Thành Phố Biên Hòa...",4.0,2,1,Đồng Nai,Thành Phố Biên Hòa


In [18]:
def clean_foody_restaurants(df):
    # Giữ lại các cột cần thiết
    df_cleaned = df.copy()
    
    # Loại bỏ các nhà hàng ở thành phố quốc tế
    international_cities = [
        'Preah Sihanouk', 'Singapore', 'Bangkok', 'Bali', 'Jakarta',
        'Western Australia', 'Phnom Penh', 'Selangor', 'New South Wales',
        'Yangon', 'Siem Reap', 'Victoria', 'Kuala Lumpur', 'California',
        'Manila', 'Kampot', 'Chon Buri', 'New York', 'Hong Kong City'
    ]
    df_cleaned = df_cleaned[~df_cleaned['city'].isin(international_cities)]
    
    # Đổi tên cột
    df_cleaned = df_cleaned.rename(columns={'city': 'city_name', 'district_raw': 'district_name'})
    
    # Chuẩn hóa city_name
    def standardize_city(city):
        if pd.isna(city):
            return "Unknown"
        
        city = str(city).strip()
        # Chuẩn hóa TP. HCM
        if city in ['TP. HCM', 'Hồ Chí Minh', 'HCM', 'TP.HCM', 'Tp. Hồ Chí Minh']:
            return 'TP. HCM'
        # Chuẩn hóa Hà Nội
        if city in ['Hà Nội', 'HN', 'Ha Noi']:
            return 'Hà Nội'
        return city
    
    # Áp dụng chuẩn hóa city_name
    df_cleaned['city_name'] = df_cleaned['city_name'].apply(standardize_city)
    
    # Tạo city_mapping dựa trên value_counts, với TP.HCM = 1 và Hà Nội = 2
    # Loại bỏ giá trị NaN và "Unknown"
    city_counts = df_cleaned['city_name'].value_counts()
    city_counts = city_counts[~city_counts.index.isin(['Unknown'])]
    
    # Sắp xếp các thành phố theo tần suất giảm dần
    sorted_cities = list(city_counts.index)
    
    # Đảm bảo TP.HCM và Hà Nội ở vị trí đầu tiên
    if 'TP. HCM' in sorted_cities:
        sorted_cities.remove('TP. HCM')
    if 'Hà Nội' in sorted_cities:
        sorted_cities.remove('Hà Nội')
    
    # Tạo danh sách thành phố theo thứ tự mong muốn
    ordered_cities = ['TP. HCM', 'Hà Nội'] + sorted_cities
    
    # Tạo city_mapping
    city_mapping = {city: idx+1 for idx, city in enumerate(ordered_cities)}
    
    # Chuẩn hóa district_name
    def standardize_district(district, city_name):
        if pd.isna(district):
            return "Unknown"
        
        district = str(district).strip().lower()
        
        # Định nghĩa mapping chuẩn hóa district
        district_standardize = {
            # TP.HCM
            'quận 1': 'quận 1', 'q.1': 'quận 1', 'q. 1': 'quận 1',
            'quận 2': 'quận 2', 'q.2': 'quận 2', 'q. 2': 'quận 2',
            'quận 3': 'quận 3', 'q.3': 'quận 3', 'q. 3': 'quận 3',
            'quận 4': 'quận 4', 'q.4': 'quận 4', 'q. 4': 'quận 4',
            'quận 5': 'quận 5', 'q.5': 'quận 5', 'q. 5': 'quận 5',
            'quận 6': 'quận 6', 'q.6': 'quận 6', 'q. 6': 'quận 6',
            'quận 7': 'quận 7', 'q.7': 'quận 7', 'q. 7': 'quận 7',
            'quận 8': 'quận 8', 'q.8': 'quận 8', 'q. 8': 'quận 8',
            'quận 9': 'quận 9', 'q.9': 'quận 9', 'q. 9': 'quận 9',
            'quận 10': 'quận 10', 'q.10': 'quận 10', 'q. 10': 'quận 10',
            'quận 11': 'quận 11', 'q.11': 'quận 11', 'q. 11': 'quận 11',
            'quận 12': 'quận 12', 'q.12': 'quận 12', 'q. 12': 'quận 12',
            'quận bình thạnh': 'quận bình thạnh', 'q. bình thạnh': 'quận bình thạnh',
            'quận tân bình': 'quận tân bình', 'q. tân bình': 'quận tân bình',
            'quận tân phú': 'quận tân phú', 'q. tân phú': 'quận tân phú',
            'quận gò vấp': 'quận gò vấp', 'q. gò vấp': 'quận gò vấp',
            'quận phú nhuận': 'quận phú nhuận', 'q. phú nhuận': 'quận phú nhuận',
            'quận bình tân': 'quận bình tân', 'q. bình tân': 'quận bình tân',
            'thủ đức': 'thành phố thủ đức', 'tp. thủ đức': 'thành phố thủ đức',
            'huyện nhà bè': 'huyện nhà bè', 'h. nhà bè': 'huyện nhà bè',
            'huyện bình chánh': 'huyện bình chánh', 'h. bình chánh': 'huyện bình chánh',
            'huyện hóc môn': 'huyện hóc môn', 'h. hóc môn': 'huyện hóc môn',
            'huyện củ chi': 'huyện củ chi', 'h. củ chi': 'huyện củ chi',
            
            # Hà Nội
            'quận ba đình': 'quận ba đình', 'q. ba đình': 'quận ba đình',
            'quận hoàn kiếm': 'quận hoàn kiếm', 'q. hoàn kiếm': 'quận hoàn kiếm',
            'quận tây hồ': 'quận tây hồ', 'q. tây hồ': 'quận tây hồ',
            'quận long biên': 'quận long biên', 'q. long biên': 'quận long biên',
            'quận cầu giấy': 'quận cầu giấy', 'q. cầu giấy': 'quận cầu giấy',
            'quận đống đa': 'quận đống đa', 'q. đống đa': 'quận đống đa',
            'quận hai bà trưng': 'quận hai bà trưng', 'q. hai bà trưng': 'quận hai bà trưng',
            'quận hoàng mai': 'quận hoàng mai', 'q. hoàng mai': 'quận hoàng mai',
            'quận thanh xuân': 'quận thanh xuân', 'q. thanh xuân': 'quận thanh xuân',
            'huyện thạch thất': 'huyện thạch thất', 'h. thạch thất': 'huyện thạch thất',
            'huyện gia lâm': 'huyện gia lâm', 'h. gia lâm': 'huyện gia lâm',
            'huyện hoài đức': 'huyện hoài đức', 'h. hoài đức': 'huyện hoài đức',
            'huyện thanh trì': 'huyện thanh trì', 'h. thanh trì': 'huyện thanh trì',
            'quận bắc từ liêm': 'quận bắc từ liêm', 'q. bắc từ liêm': 'quận bắc từ liêm',
            'quận nam từ liêm': 'quận nam từ liêm', 'q. nam từ liêm': 'quận nam từ liêm',
            'quận hà đông': 'quận hà đông', 'q. hà đông': 'quận hà đông'
        }
        
        # Kiểm tra trong mapping chuẩn hóa
        if district in district_standardize:
            return district_standardize[district]
        
        # Xử lý các pattern phổ biến
        if district.startswith('quận ') or district.startswith('q. '):
            # Giữ nguyên nếu đã có prefix đúng
            return district
        
        # Xử lý trường hợp chỉ có số (Quận X)
        if district.isdigit() and city_name == 'TP. HCM':
            return f"quận {district}"
        
        # Xử lý các prefix
        if district.startswith('tp.') or district.startswith('thành phố'):
            cleaned = district.replace('tp.', '').replace('thành phố', '').strip()
            return f"thành phố {cleaned}"
        
        if district.startswith('h.') or district.startswith('huyện'):
            cleaned = district.replace('h.', '').replace('huyện', '').strip()
            return f"huyện {cleaned}"
        
        if district.startswith('q.') or district.startswith('quận'):
            cleaned = district.replace('q.', '').replace('quận', '').strip()
            return f"quận {cleaned}"
        
        return district
    
    # Áp dụng chuẩn hóa district_name
    df_cleaned['district_name'] = df_cleaned.apply(
        lambda row: standardize_district(row['district_name'], row['city_name']), 
        axis=1
    )
    
    # Tạo city_id
    df_cleaned['city_id'] = df_cleaned['city_name'].map(city_mapping)
    df_cleaned['city_id'] = df_cleaned['city_id'].fillna(0).astype(int)
    
    # BEFood's existing district mapping
    befood_districts = {
        'huyện thạch thất': 1, 'quận 1': 2, 'quận 10': 3, 'quận 11': 4, 'quận 12': 5,
        'quận 2': 6, 'quận 3': 7, 'quận 4': 8, 'quận 5': 9, 'quận 6': 10,
        'quận 7': 11, 'quận 8': 12, 'quận 9': 13, 'quận ba đình': 14,
        'huyện bình chánh': 15, 'quận bình thạnh': 16, 'quận bình tân': 17,
        'quận bắc từ liêm': 18, 'quận cầu giấy': 19, 'huyện gia lâm': 21,
        'quận gò vấp': 22, 'quận hai bà trưng': 23, 'huyện hoài đức': 24,
        'quận hoàn kiếm': 25, 'quận hoàng mai': 26, 'quận hà đông': 27,
        'huyện hóc môn': 28, 'quận long biên': 29, 'quận nam từ liêm': 30,
        'huyện nhà bè': 31, 'quận phú nhuận': 32, 'huyện thanh trì': 33,
        'quận thanh xuân': 34, 'quận tân bình': 35, 'quận tân phú': 36,
        'quận tây hồ': 38, 'quận đống đa': 40, 'thành phố thủ đức': 41
    }
    
    # Tạo district_id bằng cách:
    # 1. Dùng befood_districts cho những quận đã biết
    # 2. Tạo district_id mới cho những quận chưa biết
    
    # Tạo district_key cho việc mapping
    df_cleaned['district_key'] = df_cleaned['district_name'].str.lower()
    
    # Tạo một dict mới để lưu tất cả district_id
    all_districts = befood_districts.copy()
    
    # Tìm id tiếp theo để gán cho district mới
    next_id = max(befood_districts.values()) + 1
    
    # Xử lý các district chưa có trong befood_districts
    unique_districts = set(df_cleaned['district_key'].dropna().unique())
    for district in unique_districts:
        if district not in all_districts and pd.notna(district):
            all_districts[district] = next_id
            next_id += 1
    
    # Map district_id
    df_cleaned['district_id'] = df_cleaned['district_key'].map(all_districts)
    df_cleaned['district_id'] = df_cleaned['district_id'].fillna(0).astype(int)
    
    # Xóa cột tạm
    df_cleaned.drop('district_key', axis=1, inplace=True)
    
    # Đảm bảo dữ liệu có kiểu dữ liệu phù hợp
    df_cleaned['restaurant_id'] = df_cleaned['restaurant_id'].astype(str)
    df_cleaned['restaurant_rating'] = pd.to_numeric(df_cleaned['restaurant_rating'], errors='coerce')
    df_cleaned['review_count'] = pd.to_numeric(df_cleaned['review_count'], errors='coerce').fillna(0).astype(int)
    
    return df_cleaned

In [19]:
foody_restaurants_cleaned = clean_foody_restaurants(foody_restaurants)


In [20]:
foody_restaurants_cleaned

,restaurant_id,restaurant_name,latitude,longitude,address,restaurant_rating,review_count,platform_id,city_name,district_name,city_id,district_id
0,2,Lẩu Nấm Thiên Nhiên Ashima - Nguyễn Đình Chiểu...,10.786070,106.697114,"35A Nguyễn Đình Chiểu, P. Đa Kao, Quận 1, TP. HCM",4.0,176,1,TP. HCM,quận 1,1,2
1,3,Nhung - Điện Biên - Test,21.383385,103.015743,"Số 851, Đường Võ Nguyên Giáp, Phường Mường Tha...",4.0,132,1,Điện Biên,thành phố điện biên phủ,55,337
2,7,Nhung qa test - Test - 1,1.000000,2.000000,"12 Nguyễn Đình Chiểu, Phường Phạm Ngũ Lão, Quậ...",4.0,111,1,TP. HCM,quận 1,1,2
3,8,Buffet Hải Sản - Café Central An Đông - Khách ...,10.757467,106.673387,"Windsor Plaza, 18 An Dương Vương, Quận 5, TP. HCM",4.0,189,1,TP. HCM,quận 5,1,9
4,11,Honey Moon - Nhà Hàng Tiệc Cưới,10.760614,106.699829,"Tầng 2, Cao ốc H3, Hoàng Diệu, Phường 6, Quận ...",4.0,10,1,TP. HCM,quận 4,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...
31163,199985,Lò Bánh Mì Lương Oanh,16.479240,107.568374,"98 Thái Phiên, P. Tây Lộc, Tp. Huế, Huế",4.0,2,1,Huế,thành phố huế,7,228
31164,199992,Swiss - Bel Resort Tuyen Lam Dalat,11.883076,108.451533,"Khu 7 - 8 Hồ Tuyền Lâm, P. 3, Thành Phố Đà Lạt...",4.0,44,1,Lâm Đồng,thành phố đà lạt,10,48
31165,199993,Siêu Thị Hàng Nhật,16.487564,107.587503,"121 Nguyễn Chí Thanh, Tp. Huế, Huế",4.0,1,1,Huế,thành phố huế,7,228
31166,199995,Bún Mắm Nêm Khuya,10.940482,106.863754,"Trần Quốc Toản, P. Bình Đa, Thành Phố Biên Hòa...",4.0,2,1,Đồng Nai,thành phố biên hòa,6,101


In [21]:
foody_reviews = pd.read_csv('/kaggle/input/restaurants-dataset/foody_final_data/foody_final_data/foody_reviews.csv')
foody_reviews

,rating_id,restaurant_id,user_id,user_name,rating,review_text,review_time,platform_id
0,18518423,2,55997891,Nữ Hoàng Remix,3,"Đồ uống rất ngon, nhân viên phục vụ nhiệt tìn...",07/07/2024,1
1,11399908,2,12759213,Ngọc Lê,4,Ăn quán từ những ngày đầu cũng chi nhánh này.\...,23/02/2021,1
2,11198395,2,9247717,Như Thùy Trần Ngọc,4,"Hương vị lẩu thơm ngon m, vị nấm ngọt có thêm ...",31/01/2021,1
3,11198259,2,15931446,Diễm Mi Nguyễn,4,Mình có may mắn nhận được voucher 500k từ Ashi...,31/01/2021,1
4,9056035,2,1320404,Michelle Phung,3,"Mình thích không gian ở đây lắm luôn, có hẳn c...",31/10/2020,1
...,...,...,...,...,...,...,...,...
469193,2011244,199999,8618389,Dã Quỳ Hoa,4,Rất thik ăn món cơm chiên heo sữa quay ở đây; ...,12/04/2018,1
469194,1901655,199999,10150742,foodee_ekuns0p7,5,"Hôm đó mình đi 2 người, vào quán điều đầu tiên...",24/01/2018,1
469195,1422797,199999,585930,Vitshym Nguyễn,4,"Quán bán cơm gà, cơm tấm, hủ tiếu, hoành thánh...",14/05/2017,1
469196,790383,199999,961914,Mino San,3,"Mình rất thích cơm gà ở đây, quán có mở thêm ở...",05/04/2016,1


In [22]:
import pandas as pd
import re
from datetime import datetime

def clean_foody_reviews(df):
    # Giữ lại các cột cần thiết
    df_cleaned = df.copy()
    
    # Đảm bảo dữ liệu có kiểu dữ liệu phù hợp
    df_cleaned['restaurant_id'] = df_cleaned['restaurant_id'].astype(str)
    df_cleaned['rating_id'] = df_cleaned['rating_id'].astype(str)
    df_cleaned['user_id'] = df_cleaned['user_id'].astype(str)
    df_cleaned['rating'] = pd.to_numeric(df_cleaned['rating'], errors='coerce')
    # Thêm tiền tố cho user_id (nếu cần) 
    df_cleaned['user_id'] = 'FU_' + df_cleaned['user_id']
    # Hàm xử lý thời gian để chuyển về định dạng dd/mm/yyyy
    def convert_foody_time(time_str):
        if pd.isna(time_str):
            return None
        
        time_str = str(time_str)
        
        # Kiểm tra nếu đã ở định dạng dd/mm/yyyy
        if re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', time_str):
            return time_str
        
        # Nếu ở định dạng yyyy-mm-dd hoặc yyyy/mm/dd hoặc các định dạng khác
        try:
            dt = pd.to_datetime(time_str)
            return dt.strftime('%d/%m/%Y')
        except:
            # Xử lý các trường hợp đặc biệt
            if time_str == 'NaT':
                return None
            return time_str  # Giữ nguyên nếu không thể chuyển đổi
    
    # Áp dụng hàm chuyển đổi thời gian
    df_cleaned['review_time'] = df_cleaned['review_time'].apply(convert_foody_time)
    
    return df_cleaned

foody_reviews_cleaned = clean_foody_reviews(foody_reviews)


In [23]:
foody_reviews_cleaned

,rating_id,restaurant_id,user_id,user_name,rating,review_text,review_time,platform_id
0,18518423,2,FU_55997891,Nữ Hoàng Remix,3,"Đồ uống rất ngon, nhân viên phục vụ nhiệt tìn...",07/07/2024,1
1,11399908,2,FU_12759213,Ngọc Lê,4,Ăn quán từ những ngày đầu cũng chi nhánh này.\...,23/02/2021,1
2,11198395,2,FU_9247717,Như Thùy Trần Ngọc,4,"Hương vị lẩu thơm ngon m, vị nấm ngọt có thêm ...",31/01/2021,1
3,11198259,2,FU_15931446,Diễm Mi Nguyễn,4,Mình có may mắn nhận được voucher 500k từ Ashi...,31/01/2021,1
4,9056035,2,FU_1320404,Michelle Phung,3,"Mình thích không gian ở đây lắm luôn, có hẳn c...",31/10/2020,1
...,...,...,...,...,...,...,...,...
469193,2011244,199999,FU_8618389,Dã Quỳ Hoa,4,Rất thik ăn món cơm chiên heo sữa quay ở đây; ...,12/04/2018,1
469194,1901655,199999,FU_10150742,foodee_ekuns0p7,5,"Hôm đó mình đi 2 người, vào quán điều đầu tiên...",24/01/2018,1
469195,1422797,199999,FU_585930,Vitshym Nguyễn,4,"Quán bán cơm gà, cơm tấm, hủ tiếu, hoành thánh...",14/05/2017,1
469196,790383,199999,FU_961914,Mino San,3,"Mình rất thích cơm gà ở đây, quán có mở thêm ở...",05/04/2016,1


In [24]:
foody_dishes

,restaurant_id,dish_type_id,dish_type_name,dish_id,dish_name,description,price,is_active,is_available,is_group_discount_item,total_like
0,1000045833,-1,Món Đang Giảm,159738838,Phật Nhảy Tường - 佛跳墙,"Hải Sâm Gai Thượng Hạng, Vi Cá, Bào Ngư, Cồi S...",398000.0,True,True,True,10+
1,1000045833,-1,Món Đang Giảm,153323321,Combo 3 Pudding Thỏ - 小兔布丁套餐🍋🍓,🍓🍋🍓,90000.0,True,True,True,2
2,1000045833,7165137,Tặng Ngay Túi Vải - Phiên Bản Giới Hạn,161546054,Tặng Ngay Túi Vải,Áp Dụng Cho Đơn Hàng Từ 299k,1.0,True,True,True,1
3,1000045833,7422548,新款热卖^-^,164956002,Rùa Quy Linh Cao Trà Sữa,乌龟龟苓膏配奶茶,29999.0,True,True,True,0
4,1000045833,2776028,大厨推荐 - Chef Recommendation,47242807,Mì Dầu Hào Cồi Sò Điệp HongKong Thượng Hạng - ...,NaN,69999.0,True,True,True,10+
...,...,...,...,...,...,...,...,...,...,...,...
110087,1027528,1151660,MÓN ĂN,45024810,"Bún Quậy Cá,Mực",Bún quậy chả cá và mực ăn kèm với rau sống và ...,75000.0,True,True,True,3
110088,1027528,1151660,MÓN ĂN,45034483,"Bún Quậy: Cá, Bò, Mực","Bún quậy 3 món: chả cá, thịt bò, mực ăn kèm ra...",85000.0,True,True,True,1
110089,1027528,1151660,MÓN ĂN,45024803,"Bún Quậy Cá, Bò",Bún quậy chả cá và thịt bò ăn kèm với rau sống...,75000.0,True,True,True,2
110090,1027528,1151661,NƯỚC UỐNG,28706635,Nước Mía,NaN,15000.0,True,True,True,10+


In [25]:
def clean_foody_dishes(df):
    # Giữ lại các cột cần thiết
    df_cleaned = df[['restaurant_id', 'dish_type_id', 'dish_type_name', 'dish_id', 'dish_name', 'price']].copy()
    
    # Đổi tên cột
    df_cleaned = df_cleaned.rename(columns={
        'dish_type_id': 'category_id',
        'dish_type_name': 'category_name',
        'dish_name': 'item_name'
    })
    
    # Thêm cột platform_id
    df_cleaned['platform_id'] = '1'
    
    # Đảm bảo dữ liệu có kiểu dữ liệu phù hợp
    df_cleaned['restaurant_id'] = df_cleaned['restaurant_id'].astype(str)
    df_cleaned['dish_id'] = df_cleaned['dish_id'].astype(str)
    df_cleaned['category_id'] = df_cleaned['category_id'].astype(str)
    df_cleaned['price'] = pd.to_numeric(df_cleaned['price'], errors='coerce')
    
    return df_cleaned

foody_dishes_cleaned = clean_foody_dishes(foody_dishes)


In [26]:
foody_dishes_cleaned

,restaurant_id,category_id,category_name,dish_id,item_name,price,platform_id
0,1000045833,-1,Món Đang Giảm,159738838,Phật Nhảy Tường - 佛跳墙,398000.0,1
1,1000045833,-1,Món Đang Giảm,153323321,Combo 3 Pudding Thỏ - 小兔布丁套餐🍋🍓,90000.0,1
2,1000045833,7165137,Tặng Ngay Túi Vải - Phiên Bản Giới Hạn,161546054,Tặng Ngay Túi Vải,1.0,1
3,1000045833,7422548,新款热卖^-^,164956002,Rùa Quy Linh Cao Trà Sữa,29999.0,1
4,1000045833,2776028,大厨推荐 - Chef Recommendation,47242807,Mì Dầu Hào Cồi Sò Điệp HongKong Thượng Hạng - ...,69999.0,1
...,...,...,...,...,...,...,...
110087,1027528,1151660,MÓN ĂN,45024810,"Bún Quậy Cá,Mực",75000.0,1
110088,1027528,1151660,MÓN ĂN,45034483,"Bún Quậy: Cá, Bò, Mực",85000.0,1
110089,1027528,1151660,MÓN ĂN,45024803,"Bún Quậy Cá, Bò",75000.0,1
110090,1027528,1151661,NƯỚC UỐNG,28706635,Nước Mía,15000.0,1


In [27]:
befood_dishes_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3064624 entries, 0 to 3064623
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   restaurant_id  object 
 1   category_id    object 
 2   category_name  object 
 3   dish_id        object 
 4   item_name      object 
 5   price          float64
 6   platform_id    int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 163.7+ MB


## Lưu cái file đã làm sạch

In [28]:
# befood
befood_restaurants_cleaned.to_csv(f'{output_dir}/befood_restaurants_cleaned.csv', index=False)
befood_reviews_cleaned.to_csv(f'{output_dir}/befood_reviews_cleaned.csv', index=False)
befood_dishes_cleaned.to_csv(f'{output_dir}/befood_dishes_cleaned.csv', index=False)
# foody
foody_restaurants_cleaned.to_csv(f'{output_dir}/foody_restaurants_cleaned.csv', index=False)
foody_reviews_cleaned.to_csv(f'{output_dir}/foody_reviews_cleaned.csv', index=False)
foody_dishes_cleaned.to_csv(f'{output_dir}/foody_dishes_cleaned.csv', index=False)

# TẠO CÁC BẢNG DỮ LIỆU KẾT HỢP 

In [29]:
# 1. Tạo bảng platform
platform_table = pd.DataFrame({
    'platform_id': ['0', '1'],
    'platform_name': ['befood', 'foody']
})

In [30]:
# 2. Kết hợp dữ liệu nhà hàng và thêm tiền tố vào ID để tránh xung đột
def merge_restaurants():
    # Thêm tiền tố vào ID nhà hàng
    befood_restaurants_cleaned['restaurant_id'] = 'B_' + befood_restaurants_cleaned['restaurant_id']
    foody_restaurants_cleaned['restaurant_id'] = 'F_' + foody_restaurants_cleaned['restaurant_id']
    
    # Kết hợp dữ liệu từ cả hai nguồn
    combined_restaurants = pd.concat([befood_restaurants_cleaned, foody_restaurants_cleaned], ignore_index=True)
    
    # Chọn các cột cho restaurant_info theo sơ đồ
    restaurants = combined_restaurants[['restaurant_id', 'restaurant_name', 'latitude', 'longitude', 
                                          'address', 'restaurant_rating', 'review_count', 'city_id', 'district_id']].copy()
    
    return restaurants

In [31]:
def create_temp_table(befood_df, foody_df):
    # Tạo các bản sao để không ảnh hưởng đến dữ liệu gốc
    befood_temp = befood_df[['restaurant_id', 'platform_id', 'restaurant_rating']].copy()
    foody_temp = foody_df[['restaurant_id', 'platform_id', 'restaurant_rating']].copy()
    
    # Kết hợp dữ liệu
    combined_temp = pd.concat([befood_temp, foody_temp], ignore_index=True)
    
    # Tạo UniqueID từ platform_id và restaurant_id - CHUYỂN ĐỔI SANG CHUỖI
    combined_temp['UniqueID'] = combined_temp['platform_id'].astype(str) + '_' + combined_temp['restaurant_id']
    
    # Sắp xếp lại các cột
    combined_temp = combined_temp[['UniqueID', 'restaurant_id', 'platform_id', 'restaurant_rating']]
    
    return combined_temp

In [32]:
# 4. Kết hợp dữ liệu món ăn
def merge_dishes():
    # Cập nhật restaurant_id trong bảng dishes để phù hợp với restaurant_info
    befood_dishes_cleaned['restaurant_id'] = 'B_' + befood_dishes_cleaned['restaurant_id']
    foody_dishes_cleaned['restaurant_id'] = 'F_' + foody_dishes_cleaned['restaurant_id']
    
    # Thêm tiền tố vào dish_id
    befood_dishes_cleaned['dish_id'] = 'B_' + befood_dishes_cleaned['dish_id']
    foody_dishes_cleaned['dish_id'] = 'F_' + foody_dishes_cleaned['dish_id']
    
    # Kết hợp dữ liệu từ cả hai nguồn
    combined_dishes = pd.concat([befood_dishes_cleaned, foody_dishes_cleaned], ignore_index=True)
    
    # Chọn các cột cho dishes theo sơ đồ
    dishes = combined_dishes[['dish_id', 'item_name', 'restaurant_id', 'category_id', 'category_name', 'price']].copy()
    
    return dishes

In [33]:
# 5. Tạo bảng users và kết hợp dữ liệu đánh giá
def create_users_and_merge_reviews():
    # Cập nhật restaurant_id trong bảng reviews để phù hợp với restaurant_info
    befood_reviews_cleaned['restaurant_id'] = 'B_' + befood_reviews_cleaned['restaurant_id']
    foody_reviews_cleaned['restaurant_id'] = 'F_' + foody_reviews_cleaned['restaurant_id']
    
    # Thêm tiền tố vào rating_id
    befood_reviews_cleaned['rating_id'] = 'BR_' + befood_reviews_cleaned['rating_id']
    foody_reviews_cleaned['rating_id'] = 'FR_' + foody_reviews_cleaned['rating_id']
    
    # Tạo bảng users từ dữ liệu đánh giá
    befood_users = befood_reviews_cleaned[['user_id', 'user_name', 'platform_id']].drop_duplicates()
    foody_users = foody_reviews_cleaned[['user_id', 'user_name', 'platform_id']].drop_duplicates()
    
    # Kết hợp dữ liệu users
    combined_users = pd.concat([befood_users, foody_users], ignore_index=True).drop_duplicates(subset=['user_id'])
    
    # Kết hợp dữ liệu reviews
    befood_reviews_final = befood_reviews_cleaned[['rating_id', 'restaurant_id', 'user_id', 'rating', 'review_text', 'review_time']].copy()
    foody_reviews_final = foody_reviews_cleaned[['rating_id', 'restaurant_id', 'user_id', 'rating', 'review_text', 'review_time']].copy()
    
    # # Đổi tên cột review_text thành feedback để phù hợp với sơ đồ
    # befood_reviews_final = befood_reviews_final.rename(columns={'review_text': 'feedback'})
    # foody_reviews_final = foody_reviews_final.rename(columns={'review_text': 'feedback'})
    
    # Kết hợp dữ liệu reviews
    combined_reviews = pd.concat([befood_reviews_final, foody_reviews_final], ignore_index=True)
    
    return combined_users, combined_reviews

In [34]:
# Thực hiện các bước kết hợp dữ liệu
restaurants_table = merge_restaurants()
temp_table = create_temp_table(befood_restaurants_cleaned, foody_restaurants_cleaned)
dishes_table = merge_dishes()
users_table, reviews_table = create_users_and_merge_reviews()

In [35]:
# Xuất các bảng đã xử lý
platform_table.to_csv(f'{output_dir}/platform_table.csv', index=False)
restaurants_table.to_csv(f'{output_dir}/restaurants_table.csv', index=False)
temp_table.to_csv(f'{output_dir}/temp_table.csv', index=False)
dishes_table.to_csv(f'{output_dir}/dishes_table.csv', index=False)
users_table.to_csv(f'{output_dir}/users_table.csv', index=False)
reviews_table.to_csv(f'{output_dir}/reviews_table.csv', index=False)